#### [ 이미지를 파일 데이터셋으로 생성 ]
- 이미지 파일의 raw data 사용
- 데이터셋 파일 : csv, json, ... 등등 개발자 선택  
- 데이터셋 파일 구성 : 라벨 + 피쳐, 피쳐 + 라벨  
- 1개 행(row)에 1개 이미지가 저장됨!

[1] 모듈 로딩 및 데이터 준비 <hr>

In [69]:
## [1-1] 모듈 로딩
import cv2							## 일반적인 이미지 파일들에서 순수 이미지 데이터 다루는 모듈듈
import matplotlib.pyplot as plt		## 시각화
import numpy as np					## 순수 이미지 데이터의 저장 형식 ndarray
import os							## 폴더, 파일, 경로 관련

In [70]:
## [1-2] 데이터 준비
IMG_DIR = '../data/image/'				## 이미지 존재 폴더
ROW, COL = 60, 60						## 이미지 리사이징 크기
DS_DIR = '../data/image/DS/'		## 학습에 사용할 이미지 저장 폴더
IMG_CSV = '../data/animal_color.csv'	## 이미지 라벨과 이미지 로우데이터 저장

DEBUG_MODE = True						## 개발 단계에서 코드 체크 용도

In [71]:
## 학습용 이미지 저장 폴더 생성
if not os.path.exists(DS_DIR):
	os.makedirs(DS_DIR)
else:
	print(f'{DS_DIR} 존재함!')

../data/image/DS/ 존재함!


In [72]:
## [1-3] 사용 이미지 파일리스트 추출
IMG_FILELIST = os.listdir(IMG_DIR)


for filename in IMG_FILELIST:
	if not os.path.isfile(IMG_DIR+filename):
		# 파일이 아닌 경우 처리-리스트에서 제거
		IMG_FILELIST.remove(filename)

if DEBUG_MODE: print(IMG_FILELIST)

['apple.png', 'copy_orange.png', 'cropped.jpg', 'fish.jpg', 'girl.jpg', 'opencv_logo.png', 'orange.png', 'resize_apple.png', 'strawberry.png', 'sunset.jpg', '스크린샷 2025-03-18 114847.png']


In [73]:
aa = [12, 3, 4, 5]
aa.remove(3)
print(aa)

[12, 4, 5]


In [74]:
print(IMG_FILELIST)

['apple.png', 'copy_orange.png', 'cropped.jpg', 'fish.jpg', 'girl.jpg', 'opencv_logo.png', 'orange.png', 'resize_apple.png', 'strawberry.png', 'sunset.jpg', '스크린샷 2025-03-18 114847.png']


In [75]:
for filename in IMG_FILELIST:
	## 파일명에서 확장자 추출
	idxValue=filename.find('.')
	if idxValue>0:
		if filename[	idxValue+1: ] not in ['jpg', 'jpeg', 'png', 'bmp']:
			IMG_FILELIST.remove(filename)
	else:
		print(f'NOT FILE : {filename}')
		IMG_FILELIST.remove(filename)
	
print(IMG_FILELIST)

['apple.png', 'copy_orange.png', 'cropped.jpg', 'fish.jpg', 'girl.jpg', 'opencv_logo.png', 'orange.png', 'resize_apple.png', 'strawberry.png', 'sunset.jpg', '스크린샷 2025-03-18 114847.png']


[2] 이미지 데이터 로딩 <hr>

In [76]:
## Grayscale 형식으로 이미지 데이터 로딩
with open(IMG_CSV, mode = 'a') as csvF:
	for filename in IMG_FILELIST:
		# - 3채널로 이미지 로딩
		rgbIMG = cv2.imread(IMG_DIR+filename, cv2.IMREAD_COLOR)
		print(f'rgbIMG => {rgbIMG.shape}')

		# - 사이즈 조정
		rgbIMG = cv2.resize(rgbIMG, (COL, ROW))
		print(f'rgbIMG => {rgbIMG.shape}')

		# - 파일로 저장
		resizeFile = f'{DS_DIR}resize_{filename}'
		print(f'resizefile => {resizeFile}')
		
		# - opencv 채널 순서 BGR ==> 일반 RGB
		ret = cv2.imwrite(resizeFile, cv2.cvtColor(rgbIMG, cv2.COLOR_BGR2RGB))
		if not ret: print(f'Fail => {resizeFile}')

		# - 채널 분리
		b, g, r = cv2.split(rgbIMG)
		b, g, r = b.reshape(-1), g.reshape(-1), r.reshape(-1)

		# CSV 파일 쓰기 => 라벨, 픽셀1, 픽셀2, ....., 픽셀n
		label = filename[:3]
		data = np.concatenate((r, g, b))	## 3채널 1개로 연결
		print(f'label => {label}, data => {data.shape}')

		sdata = list(map(lambda n: str(n), data))
		print(f'sdata => {sdata[:30]}')
		
		csvF.write(str(label)+",")					# 숫자 라벨 쓰기
		csvF.write(','.join(sdata) + "\n")			# 리스트 이미지 데이터 => 문자열 변환

		break

rgbIMG => (601, 924, 3)
rgbIMG => (60, 60, 3)
resizefile => ../data/image/DS/resize_apple.png
label => app, data => (10800,)
sdata => ['252', '251', '251', '252', '250', '250', '251', '251', '251', '251', '251', '251', '251', '251', '251', '251', '251', '251', '250', '250', '249', '250', '250', '250', '250', '250', '250', '250', '250', '250']


In [77]:
import pandas as pd

In [78]:
colorDF = pd.read_csv(IMG_CSV, header=None)
colorDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Columns: 10801 entries, 0 to 10800
dtypes: int64(10800), object(1)
memory usage: 253.3+ KB


In [79]:
colorDF.head(5)

,0,1,2,3,4,5,6,7,8,9,...,10791,10792,10793,10794,10795,10796,10797,10798,10799,10800
0,app,252,251,251,252,250,250,251,251,251,...,245,245,245,245,245,246,246,246,246,246
1,app,252,251,251,252,250,250,251,251,251,...,245,245,245,245,245,246,246,246,246,246
2,app,252,251,251,252,250,250,251,251,251,...,245,245,245,245,245,246,246,246,246,246


사용자 정의 함수 <hr>

In [80]:
## 폴더 내 파일 리스트 로딩
import os

filelist = os.listdir(DATA_DIR)

NameError: name 'DATA_DIR' is not defined